<a href="https://colab.research.google.com/github/carlosdgerez/machine_learning/blob/main/module3Boost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')

In [2]:
# download the income data

zip_code = pd.read_csv('https://raw.githubusercontent.com/carlosdgerez/machine_learning/main/mean_income_washington_by_zipcode.csv')
zip_code.head()



,zip_code,mean_income
0,98001,102586
1,98002,70945
2,98003,76753
3,98004,210129
4,98005,186020


In [3]:
# rename the columns as the one in clean data to use merge by this column
zip_code.rename(columns={'zip_code': 'zipcode'}, inplace = True)

In [4]:
# download the income data

price_index = pd.read_csv('https://raw.githubusercontent.com/carlosdgerez/machine_learning/main/price_index.csv')
price_index.head()

,zipcode,Annual Change (%),HPI,HPI with 1990 base,HPI with 2000 base
0,98001,16.18,1175.72,378.94,255.89
1,98002,15.31,1061.13,424.05,263.15
2,98003,17.35,1241.23,393.20,274.01
3,98004,-10.87,1876.50,473.89,273.74
4,98005,-0.55,1646.14,467.74,288.62


In [5]:

zipcode = pd.read_csv('https://raw.githubusercontent.com/Kate-Strydom/cse450/main/notebooks/zipcode_washington.csv')

zipcode.head()

,zipcode,population,median_household_income
0,98001,31742,72540
1,98002,34240,44930
2,98003,45632,45563
3,98004,31254,100344
4,98005,17880,85639


In [6]:
#Changing date to be in the same format as the rest of the years features
import datetime
housing['date'] = pd.DatetimeIndex(housing['date']).year



In [7]:
# drop id
clean_data = housing.drop(['id'], axis = 1)

In [8]:

# change date name to something more descriptive
clean_data.rename(columns={'date': 'sale_date'}, inplace=True)

In [9]:
# Use merge to get the data in the dataframe
new_clean_data = pd.merge(clean_data, zip_code, how= 'left', on= 'zipcode')

In [10]:
# reorder the data to have the target price at the end
column_to_move = new_clean_data.pop("mean_income")

# insert column with insert(location, column_name, column_value)

new_clean_data.insert(19, "mean_income", column_to_move)


In [11]:
# Use merge to get the data in the dataframe
new_clean_data2 = pd.merge(new_clean_data, price_index, how= 'left', on= 'zipcode')

In [12]:
# reorder the data to have the target price at the end
column_to_move = new_clean_data2.pop("price")

# insert column with insert(location, column_name, column_value)

new_clean_data2.insert(24, "price", column_to_move)


In [13]:
# Use merge to get the data in the dataframe
new_clean_data3 = pd.merge(new_clean_data, zipcode, how= 'left', on= 'zipcode')

In [14]:
# reorder the data to have the target price at the end
column_to_move = new_clean_data3.pop("price")

# insert column with insert(location, column_name, column_value)

new_clean_data3.insert(22, "price", column_to_move)


In [15]:
# get a training set: a X as with the features to relate and a y with the  target 
X = new_clean_data2.iloc[:,0:-1]
y = new_clean_data2.price


In [16]:
# Start spliting the data and import libraries

#import train split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=.30, random_state=1)


In [17]:
#  xgboost regresor
import xgboost
from sklearn import metrics as sm

# createn instance and fit the model

clf2 = xgboost.XGBRegressor(random_state=5, max_depth= 4, n_estimators= 900, learning_rate= 0.1, verbosity = 0, n_jobs = -1).fit(X_train, y_train)
y_pred = clf2.predict(X_test)

# This is how to generate a regresion report ( for regresion )
print("Mean absolute error in dollars=", round(
    sm.mean_absolute_error(y_test, y_pred), 2))
print("Mean squared error =", round(
    sm.mean_squared_error(y_test, y_pred), 2))
print("Median absolute error =", round(
    sm.median_absolute_error(y_test, y_pred), 2))
print("Explain variance score =", round(
    sm.explained_variance_score(y_test, y_pred), 2))
print("R2 score =", round( sm.r2_score(y_test, y_pred),4))
print( "ROOT Mean squared error in dollars = ", 
    round(sm.mean_squared_error(y_test, y_pred, squared=False 
)))


Mean absolute error in dollars= 61570.64
Mean squared error = 12337913661.56
Median absolute error = 36403.17
Explain variance score = 0.91
R2 score = 0.9075
ROOT Mean squared error in dollars =  111076


In [18]:


#  EVALUATION OF THE MODEL 

#Predict the response for test dataset


# Features importances in a model this block can be run with diferent models to compare


feat_imports = (pd.DataFrame({"feature names": X_train.columns,
                              "importances": clf2.feature_importances_})
                .sort_values("importances", ascending=False))

feat_imports.reset_index(drop= True, inplace= True)



In [19]:
import altair as alt

feat_imports['percents'] = round((feat_imports.importances * 100).astype(float),2)
alt.Chart(feat_imports.head(20)).mark_bar().encode(
    alt.X("importances", axis=alt.Axis(format='%')),
    alt.Y('feature names',sort='-x'),
    tooltip=('percents'))

alt.Chart(...)